# Ingress Farming Route Path Finder

This is intended to find the shortest path with highest portal hack count.

## TODO:
[ ] Automate

In [2]:
import os
import json
import googlemaps

gmaps = googlemaps.Client(open('/Users/gaulinmp/.googlemaps.api').read())#key="PASTE YOUR API KEY HERE"

import networkx as nx
print("NetworkX: {}".format(nx.__version__))
import numpy as np
print("NumPy: {}".format(np.__version__))
import pandas as pd
print("Pandas: {}".format(pd.__version__))
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
print("Matplotlib: {}".format(mpl.__version__))
import smopy
print("Smopy: {}".format(smopy.__version__))
import folium
print("Folium: {}".format(folium.__version__))
mpl.rcParams['figure.dpi'] = mpl.rcParams['savefig.dpi'] = 300
from IPython.display import HTML

NetworkX: 1.9.1
NumPy: 1.9.1
Pandas: 0.15.2
Matplotlib: 1.4.2
Smopy: 0.0.3
Folium: 0.1.3


In [15]:
def inline_map(map):
    """
    Embeds the HTML source of the map directly into the IPython notebook.
    
    This method will not work if the map depends on any files (json data). Also this uses
    the HTML5 srcdoc attribute, which may not be supported in all browsers.
    """
    map._build_map()
    return HTML('<iframe srcdoc="{srcdoc}" style="width: 100%; height: 510px; border: none"></iframe>'.format(
                srcdoc=map.HTML.replace('"', '&quot;')))

In [16]:
def filter_nodes(nodes, topright, botleft):
    """Coordinates in {lat,lng} dict format."""
    for x in nodes:
        if ((topright['lat'] >= x['lat'] >= botleft['lat']) and
            (topright['lng'] >= x['lng'] >= botleft['lng'])):
            yield x
TOPRIGHT = {'lat':29.746327, 'lng':-95.390126}
BOTLEFT = {'lat':29.720117, 'lng':-95.420254}

In [17]:
GREEN = 0
BLUE = 1

In [18]:
def get_portal_coords(json_object):
    portals = []
    for loc,val in json_object['result']['map'].items():
        if 'gameEntities' not in val: continue
        for portal_raw in val['gameEntities']:
            tmp = {}
            try: 
                src = portal_raw[2]
            except:
                continue
            if 'type' not in src or src['type'] != 'portal':
                continue
            tmp['level'] = src['level']
            tmp['resonators'] = src['resCount']
            tmp['team'] = GREEN if src['team']=='ENLIGHTENED' else BLUE
            tmp['lat'] = src['latE6'] / 10**6
            tmp['lng'] = src['lngE6'] / 10**6
            tmp['name'] = src['title']
            tmp['img'] = src['image']
            portals.append(tmp)
    return portals
tmp = get_portal_coords(json.loads(open('./getElements_dump_77098.json').read()))
print(tmp[0])

{'name': 'Moran Center', 'lat': 29.738539, 'img': 'http://lh6.ggpht.com/0LnmfQtfYS3VS20LsmaiYgmO_ACKMj0iFzK_JI_I_PyQEgL1jP7JbMvs4bJxy0oxpR-mX--lH9BL6z8h_3U0Jg', 'resonators': 3, 'team': 0, 'level': 3, 'lng': -95.394638}


In [19]:
home = {'lat':29.738632, 'lng': -95.404546} # 29.738632, -95.404546

folmap = folium.Map(location=[home['lat'], home['lng']], zoom_start=15)
#map.simple_marker([home['lat'], home['lng']])
for marker in filter_nodes(tmp, TOPRIGHT, BOTLEFT):
    folmap.circle_marker([marker['lat'], marker['lng']]
                         ,popup="{name}<br /><img src='{img}' alt='{name}' width=100></img>".format(**marker)
                         ,radius=marker['level']*3
                         ,fill_color='#0000ff', line_color='#0000ff')
#for x in filter_nodes([{'lat': x[1], 'lng':x[0]} for x in g.nodes()], TOPRIGHT, BOTLEFT):
#    map.circle_marker([x['lat'], x['lng']], radius=10)
for x in tmp:
    folmap.circle_marker(location=[x['lat'], x['lng']], radius=10
                         ,fill_color='#222222', line_color='#00ffffff', popup=str([x['lat'], x['lng']]))
inline_map(folmap)

In [129]:
from pyroutelib2.loadOsm import LoadOsm

# Open Street Map API:
* http://api.openstreetmap.org//api/0.6/map?bbox=-95.420254,29.720117,-95.390126,29.746327
* API: https://github.com/metaodi/osmapi/blob/develop/osmapi/OsmApi.py
* TIGER: http://wiki.openstreetmap.org/wiki/TIGER_to_OSM_Attribute_Map

In [133]:
import osmapi
api = osmapi.OsmApi()
api._debug = True

In [131]:
def get_osm_nodes_ways(api, bot_left, top_right):
    """Returns (nodes, ways, osm_data) in bounding box."""
    data = api.Map(bot_left['lng'], bot_left['lat'], top_right['lng'], top_right['lat'])
    nodes, ways, type_dict = {}, {}, None
    for x in data:
        try:
            if x['type'] == 'node':
                nodes[x['data']['id']] = x['data']
            elif x['type'] == 'way':
                if 'highway' in x['data']['tag']:
                    ways[x['data']['id']] = x['data']
            else:
                continue
        except KeyError:
            # Don't care about bad data (no type/data key)
            continue
    return nodes, ways, data

In [134]:
nodes, ways, osm_data = get_osm_nodes_ways(api, BOTLEFT, TOPRIGHT)
street_nodes = {}
for wayid, wayinfo in ways.items():
    for nds in wayinfo['nd']:
        if nds not in nodes:
            continue
        street_nodes[nds] = (nodes[nds])
street_nodes = [val for key,val in street_nodes.items()]

2015-01-07 00:52:39 GET /api/0.6/map?bbox=-95.420254,29.720117,-95.390126,[...]
2015-01-07 00:52:47 GET /api/0.6/map?bbox=-95.420254,29.720117,-95.390126,[...]


In [140]:
import os
try:  # Python 3.x
  from urllib.request import urlretrieve
except ImportError:  # Python 2.x
  from urllib import urlretrieve

def GetOsmTileData(bot_left_lat, bot_left_lng, top_right_lat, top_right_lng):
  """Download OSM data for the region covering:
     (bot_left_lat,bot_left_lng) to (top_right_lat, top_right_lng)"""
  if(bot_left_lat >= top_right_lat or bot_left_lng >= top_right_lng):
    print("Bottom Left coords must be less than Top Right.")
    return
  
  directory = 'cache/{:0.2f}-{:0.2f}_{:0.2f}-{:0.2f}'.format(bot_left_lat, bot_left_lng, top_right_lat, top_right_lng)
  print(directory)  
  filename = '%s/data.osm' % (directory)
  if(not os.path.exists(directory)):
    os.makedirs(directory)
  URL = 'http://api.openstreetmap.org/api/0.6/map?bbox={},{},{},{}'.format(
         bot_left_lng, bot_left_lat, top_right_lng, top_right_lat)
  print(URL)
  if(not os.path.exists(filename)): # TODO: allow expiry of old data
    urlretrieve(URL, filename)
  return(filename)
    
print(GetOsmTileData(BOTLEFT['lat'], BOTLEFT['lng'], TOPRIGHT['lat'], TOPRIGHT['lng']))

cache/-95.42-29.72_-95.39-29.75
http://api.openstreetmap.org/api/0.6/map?bbox=-95.420254,29.720117,-95.390126,29.746327
cache/-95.42-29.72_-95.39-29.75/data.osm


In [149]:
[x for x in osm_data if x['type']=='way']

[{'data': {'user': 'afdreher',
   'tag': {'tiger:zip_right': '77005',
    'tiger:zip_left': '77005',
    'tiger:cfcc': 'A41',
    'name': 'Remington Lane',
    'tiger:county': 'Harris, TX',
    'tiger:name_base': 'Remington',
    'highway': 'residential',
    'access': 'private',
    'tiger:name_type': 'Ln'},
   'changeset': 15742004,
   'uid': 1110270,
   'nd': [152216469,
    2268654112,
    2268654128,
    152216471,
    152216474,
    152216476,
    152216478,
    152216480,
    152216481,
    152216482,
    152216483,
    152216484,
    152216485,
    152216486,
    152216487,
    151820458,
    152216489,
    152216490,
    152216491],
   'id': 15364251,
   'version': 4,
   'timestamp': '2013-04-15T20:32:43Z',
   'visible': True},
  'type': 'way'},
 {'data': {'user': 'afdreher',
   'tag': {'tiger:zip_right': '77005',
    'tiger:zip_left': '77005',
    'tiger:cfcc': 'A41',
    'name': 'Longfellow Lane',
    'tiger:county': 'Harris, TX',
    'tiger:name_base': 'Longfellow',
    'hi

# A* algorithm
From [here](http://nbviewer.ipython.org/github/rhiever/Data-Analysis-and-Machine-Learning-Projects/blob/master/optimal-road-trip/Computing%20the%20optimal%20road%20trip%20across%20the%20U.S..ipynb)

In [20]:
tmp[0]

{'img': 'http://lh6.ggpht.com/0LnmfQtfYS3VS20LsmaiYgmO_ACKMj0iFzK_JI_I_PyQEgL1jP7JbMvs4bJxy0oxpR-mX--lH9BL6z8h_3U0Jg',
 'lat': 29.738539,
 'level': 3,
 'lng': -95.394638,
 'name': 'Moran Center',
 'resonators': 3,
 'team': 0}

In [21]:
import math

def map_distance(x,y):
    """ Input x and y as objects with 'lat' and 'lng' coordinates. """
    try:
        lat1, lng1 = x['lat'], x['lng']
        lat2, lng2 = y['lat'], y['lng']
    except:
        raise ValueError('')
    R = 6373370
    lng1, lat1, llng2, lat2 = map(math.radians, [lng1, lat1, lng2, lat2])
    # haversine formula 
    dlng = lng2 - lng1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlng/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    return R * c

In [22]:
map_distance(tmp[0], tmp[1])

2883818.1542073186

In [43]:
from itertools import combinations

waypoint_distances = {}
waypoint_durations = {}
walking_speed = 5000/3600 # m / h * h / 3600 s

for (x, y) in combinations(tmp, 2):
    try:
        # "distance" is in meters
        distance = map_distance(x, y)
        # "duration" is in seconds
        duration = distance / walking_speed # m / (m/s) = s
        tmp_hash = frozenset([googlemaps.convert.latlng(x), googlemaps.convert.latlng(y)])
        try:
            a,b = tmp_hash
        except:
            continue
        waypoint_distances[tmp_hash] = distance
        waypoint_durations[tmp_hash] = duration
    
    except UnicodeDecodeError as e:
        print("Error with finding the route between %s and %s." % (x, y))

In [44]:
with open("my-waypoints-dist-dur.tsv", "w") as out_file:
    out_file.write("\t".join(["waypoint1",
                              "waypoint2",
                              "distance_m",
                              "duration_s"]))
    for (waypoint1, waypoint2) in waypoint_distances.keys():
        out_file.write("\n" +
                       "\t".join([waypoint1,
                                  waypoint2,
                                  str(waypoint_distances[frozenset([waypoint1, waypoint2])]),
                                  str(waypoint_durations[frozenset([waypoint1, waypoint2])])]))

In [ ]:
import pandas as pd
import numpy as np

waypoint_distances = {}
waypoint_durations = {}
all_waypoints = set()

waypoint_data = pd.read_csv("my-waypoints-dist-dur.tsv", sep="\t")

for i, row in waypoint_data.iterrows():
    waypoint_distances[frozenset([row.waypoint1, row.waypoint2])] = row.distance_m
    waypoint_durations[frozenset([row.waypoint1, row.waypoint2])] = row.duration_s
    all_waypoints.update([row.waypoint1, row.waypoint2])

In [ ]:
import random

def compute_fitness(solution):
    """
        This function returns the total distance traveled on the current road trip.
        
        The genetic algorithm will favor road trips that have shorter
        total distances traveled.
    """
    
    solution_fitness = 0.0
    
    for index in range(len(solution)):
        waypoint1 = solution[index - 1]
        waypoint2 = solution[index]
        solution_fitness += waypoint_distances[frozenset([waypoint1, waypoint2])]
        
    return solution_fitness

def generate_random_agent():
    """
        Creates a random road trip from the waypoints.
    """
    
    new_random_agent = list(all_waypoints)
    random.shuffle(new_random_agent)
    return tuple(new_random_agent)

def mutate_agent(agent_genome, max_mutations=3):
    """
        Applies 1 - `max_mutations` point mutations to the given road trip.
        
        A point mutation swaps the order of two waypoints in the road trip.
    """
    
    agent_genome = list(agent_genome)
    num_mutations = random.randint(1, max_mutations)
    
    for mutation in range(num_mutations):
        swap_index1 = random.randint(0, len(agent_genome) - 1)
        swap_index2 = swap_index1

        while swap_index1 == swap_index2:
            swap_index2 = random.randint(0, len(agent_genome) - 1)

        agent_genome[swap_index1], agent_genome[swap_index2] = agent_genome[swap_index2], agent_genome[swap_index1]
            
    return tuple(agent_genome)

def shuffle_mutation(agent_genome):
    """
        Applies a single shuffle mutation to the given road trip.
        
        A shuffle mutation takes a random sub-section of the road trip
        and moves it to another location in the road trip.
    """
    
    agent_genome = list(agent_genome)
    
    start_index = random.randint(0, len(agent_genome) - 1)
    length = random.randint(2, 20)
    
    genome_subset = agent_genome[start_index:start_index + length]
    agent_genome = agent_genome[:start_index] + agent_genome[start_index + length:]
    
    insert_index = random.randint(0, len(agent_genome) + len(genome_subset) - 1)
    agent_genome = agent_genome[:insert_index] + genome_subset + agent_genome[insert_index:]
    
    return tuple(agent_genome)

def generate_random_population(pop_size):
    """
        Generates a list with `pop_size` number of random road trips.
    """
    
    random_population = []
    for agent in range(pop_size):
        random_population.append(generate_random_agent())
    return random_population
    
def run_genetic_algorithm(generations=5000, population_size=100):
    """
        The core of the Genetic Algorithm.
    """
    
    # Create a random population of `population_size` number of solutions.
    population = generate_random_population(population_size)

    # For `generations` number of repetitions...
    for generation in range(generations):
        
        # Compute the fitness of the entire current population
        population_fitness = {}

        for agent_genome in population:
            if agent_genome in population_fitness:
                continue

            population_fitness[agent_genome] = compute_fitness(agent_genome)

        # Take the 10 shortest road trips and produce 10 offspring each from them
        new_population = []
        for rank, agent_genome in enumerate(sorted(population_fitness, key=population_fitness.get)[:10]):
            if (generation % 1000 == 0 or generation == generations - 1) and rank == 0:
                print("Generation %d best: %d | Unique genomes: %d" % (generation,
                                                                       population_fitness[agent_genome],
                                                                       len(population_fitness)))
                print(agent_genome)
                print("")

            # Create 1 exact copy of each of the top 10 road trips
            new_population.append(agent_genome)

            # Create 2 offspring with 1-3 point mutations
            for offspring in range(2):
                new_population.append(mutate_agent(agent_genome, 3))
                
            # Create 7 offspring with a single shuffle mutation
            for offspring in range(7):
                new_population.append(shuffle_mutation(agent_genome))

        # Replace the old population with the new population of offspring 
        for i in range(len(population))[::-1]:
            del population[i]

        population = new_population

In [ ]:
run_genetic_algorithm(generations=5000, population_size=100)